In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [3]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = False, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Mllama vision patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [4]:
# model = FastVisionModel.get_peft_model(
#     model,
#     finetune_vision_layers     = True, # False if not finetuning vision layers
#     finetune_language_layers   = True, # False if not finetuning language layers
#     finetune_attention_modules = True, # False if not finetuning attention layers
#     finetune_mlp_modules       = True, # False if not finetuning MLP layers

#     r = 16,           # The larger, the higher the accuracy, but might overfit
#     lora_alpha = 16,  # Recommended alpha == r at least
#     lora_dropout = 0,
#     bias = "none",
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
#     # target_modules = "all-linear", # Optional now! Can specify a list if needed
# )

In [7]:
type(model)

transformers.models.mllama.modeling_mllama.MllamaForConditionalGeneration

In [8]:
type(tokenizer)

transformers.models.mllama.processing_mllama.MllamaProcessor

# Test Inference

In [9]:
!git clone https://github.com/heng2j/CompetitiveSketchAgent.git

Cloning into 'CompetitiveSketchAgent'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 151 (delta 68), reused 79 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (151/151), 431.15 KiB | 1.06 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [10]:
%cd CompetitiveSketchAgent
!git checkout EDA_with_Llama_vision
!pip install -e .
!pip list | grep competitive
!pip install cairosvg

/content/CompetitiveSketchAgent
Branch 'EDA_with_Llama_vision' set up to track remote branch 'EDA_with_Llama_vision' from 'origin'.
Switched to a new branch 'EDA_with_Llama_vision'
Obtaining file:///content/CompetitiveSketchAgent
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for competitive_sketch_agent (pyproject.toml) ... done
  Created wheel for competitive_sketch_agent: filename=competitive_sketch_agent-2024.12-py3-none-any.whl size=2144 sha256=006ccaebdacd95171819a8cdf7597762efdfc9baca35716e0b4809cd4720a2c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-c_82kaze/wheels/33/4d/2c/820e3761f4a29dd65d20fa1447c74ba166a5e4dda3c911efaf
Successfully built competitive_sketch_agent
competitive_sketch_agent           2024.12              /content/CompetitiveSketchAgent
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
from competitive_sketch_agent import utils
from competitive_sketch_agent.prompts import sketch_first_prompt, system_prompt, gt_example
from competitive_sketch_agent.sketch_generator import SketchApp

import datetime
import os

In [12]:
filename = "/content/CompetitiveSketchAgent/competitive_sketch_agent/configs/default_config.yaml"
config = utils.load_yaml_config(filename=filename)

In [13]:
sketch_app = SketchApp(config=config)
sketch_app.grid_size = (612,612)

In [14]:
sketch_app.input_prompt

"I provide you with a blank grid. Your goal is to produce a visually appealing sketch of a car.\nHere are a few examples:\n<examples>\n\n<example>\nTo draw a house, start by drawing the front of the house:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>\n        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>\n        <id>house base front rectangle</id>\n    </s1>\n    <s2>\n        <points>'x13y27', 'x18y37','x18y37', 'x24y27'</points>\n        <t_values>0.00,0.55,0.5,1.00</t_values>\n        <id>roof front triangle</id>\n    </s2>\n</strokes>\n\nNext we add the house's right section:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>\n        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>\n        <id>house base front rectangle</id>\n    </s1>\n    <s2>\n       

In [15]:
system_prompt

'You are an expert artist specializing in drawing sketches that are visually appealing, expressive, and professional.\nYou will be provided with a blank grid. Your task is to specify where to place strokes on the grid to create a visually appealing sketch of the given textual concept.\nThe grid uses numbers (1 to {res}) along the bottom (x axis) and numbers (1 to {res}) along the left edge (y axis) to reference specific locations within the grid. Each cell is uniquely identified by a combination of the corresponding x axis numbers and y axis number (e.g., the bottom-left cell is \'x1y1\', the cell to its right is \'x2y1\').\nYou can draw on this grid by specifying where to draw strokes. You can draw multiple strokes to depict the whole object, where different strokes compose different parts of the object. \nTo draw a stroke on the grid, you need to specify the following:\nStarting Point: Specify the starting point by giving the grid location (e.g., \'x1y1\' for column 1, row 1).\nEndin

In [16]:
sketch_app.res

50

In [17]:
msg_history=[]
init_canvas_str=None
msg=sketch_app.input_prompt
other_msg = sketch_app.define_input_to_llm(msg_history, sketch_app.init_canvas_str, msg)
other_msg


[{'role': 'user',
  'content': [{'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/jpeg',
     'data': '...'}},
   {'type': 'text',
    'text': "I provide you with a blank grid. Your goal is to produce a visually appealing sketch of a car.\nHere are a few examples:\n<examples>\n\n<example>\nTo draw a house, start by drawing the front of the house:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>\n        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>\n        <id>house base front rectangle</id>\n    </s1>\n    <s2>\n        <points>'x13y27', 'x18y37','x18y37', 'x24y27'</points>\n        <t_values>0.00,0.55,0.5,1.00</t_values>\n        <id>roof front triangle</id>\n    </s2>\n</strokes>\n\nNext we add the house's right section:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11',

In [18]:
type(sketch_app.init_canvas)

PIL.Image.Image

In [19]:
FastVisionModel.for_inference(model) # Enable for inference!


messages = other_msg


In [ ]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
image = sketch_app.init_canvas

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
response = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 3000,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

<answer>
<concept>A house</concept>
<strokes>
    <s1>
        <points>'x13y27', 'x13y27', 'x13y33', 'x24y27', 'x24y27', 'x24y33', 'x13y33'</points>
        <t_values>0.0, 0.2, 0.3, 0.5, 0.6, 0.75, 1.00</t_values>
        <id>bottom section</id>
    </s1>
    <s2>
        <points>'x13y33', 'x24y33', 'x24y27', 'x13y27'</points>
        <t_values>0.0, 0.55, 0.7, 1.00</t_values>
        <id>front wall</id>
    </s2>
    <s3>
        <points>'x24y33', 'x36y38', 'x36y38', 'x24y37', 'x24y37', 'x24y27', 'x24y27', 'x36y27'</points>
        <t_values>0.0, 0.3, 0.25, 0.5, 0.6, 0.75, 0.7, 1.00</t_values>
        <id>left roof</id>
    </s3>
    <s4>
        <points>'x13y33', 'x24y37', 'x24y33', 'x36y38', 'x36y38'</points>
        <t_values>0.0, 0.3, 0.25, 0.5, 0.55</t_values>
        <id>front roof</id>
    </s4>
</strokes>
</answer><|eot_id|>


In [ ]:
  # 3. Decode
  output_text = tokenizer.decode(
      response[0],
      skip_special_tokens=True
  )

  output_text

"user\n\nI provide you with a blank grid. Your goal is to produce a visually appealing sketch of a house.\nHere are a few examples:\n<examples>\n\n<example>\nTo draw a house, start by drawing the front of the house:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>\n        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>\n        <id>house base front rectangle</id>\n    </s1>\n    <s2>\n        <points>'x13y27', 'x18y37','x18y37', 'x24y27'</points>\n        <t_values>0.00,0.55,0.5,1.00</t_values>\n        <id>roof front triangle</id>\n    </s2>\n</strokes>\n\nNext we add the house's right section:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>\n        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>\n        <id>house base front rectangle</id>\n    </s1>\n    <s2

In [ ]:
sketching_commands = output_text



model_strokes_svg = sketch_app.parse_model_to_svg(sketching_commands)
current_direcotory = os.getcwd()
out_path = os.path.join(current_direcotory ,sketch_app.path2save, sketch_app.target_concept)
os.makedirs(out_path, exist_ok=True)


with open(f"{sketch_app.path2save}/{sketch_app.target_concept}.svg", "w") as svg_file:
    svg_file.write(model_strokes_svg)


# # vector->pixel
# # save the sketch to png with blank backgournd
# cairosvg.svg2png(url=f"{self.path2save}/{self.target_concept}.svg", write_to=f"{self.path2save}/{self.target_concept}.png", background_color="white")

# # save the sketch to png on the canvas
# output_png_path = f"{self.path2save}/{self.target_concept}_canvas.png"
# cairosvg.svg2png(url=f"{self.path2save}/{self.target_concept}.svg", write_to=output_png_path)
# foreground = Image.open(output_png_path)
# self.init_canvas.paste(Image.open(output_png_path), (0, 0), foreground)
# self.init_canvas.save(output_png_path)

In [ ]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
image = sketch_app.init_canvas

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
response = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 3000,
                   use_cache = True, temperature = 1.0, min_p = 0.1)

The concept depicted in the sketch is a house.

The strokes that define the house are as follows:

<s1> The house base front rectangle.
<s2> The roof front triangle.
<s3> The house base right section.
<s4> The roof right section.
<s5> The left window square.
<s6> The right window square.
<s7> The front door.

The x-y coordinates that define each stroke are as follows:

<s1>
<points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>
<t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
<id>house base front rectangle</id>
</s1>

<s2>
<points>'x13y27', 'x18y37','x18y37', 'x24y27'</points>
<t_values>0.00,0.55,0.5,1.00</t_values>
<id>roof front triangle</id>
</s2>

<s3>
<points>'x24y27', 'x36y28', 'x36y28', 'x36y21', 'x36y21', 'x36y12', 'x36y12', 'x24y11'</points>
<t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
<id>house base right section</id>
</s3>

<s4>
<points>'x18y37', 'x30y38', 'x30y38', 'x36y28'</points>
<t_values>0.00,0.55,0.5,1.00<

In [ ]:
response

tensor([[128000, 128006,    882,  ...,  56137,     29, 128009]],
       device='cuda:0')

In [ ]:
  # 3. Decode
  output_text = tokenizer.decode(
      response[0],
      skip_special_tokens=True
  )

  output_text

"user\n\nI provide you with a blank grid. Your goal is to produce a visually appealing sketch of a house.\nHere are a few examples:\n<examples>\n\n<example>\nTo draw a house, start by drawing the front of the house:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>\n        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>\n        <id>house base front rectangle</id>\n    </s1>\n    <s2>\n        <points>'x13y27', 'x18y37','x18y37', 'x24y27'</points>\n        <t_values>0.00,0.55,0.5,1.00</t_values>\n        <id>roof front triangle</id>\n    </s2>\n</strokes>\n\nNext we add the house's right section:\n<concept>House</concept>\n<strokes>\n    <s1>\n        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>\n        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>\n        <id>house base front rectangle</id>\n    </s1>\n    <s2

In [ ]:
sketching_commands = output_text



model_strokes_svg = sketch_app.parse_model_to_svg(sketching_commands)
current_direcotory = os.getcwd()
out_path = os.path.join(current_direcotory ,sketch_app.path2save, sketch_app.target_concept)
os.makedirs(out_path, exist_ok=True)


with open(f"{sketch_app.path2save}/{sketch_app.target_concept}.svg", "w") as svg_file:
    svg_file.write(model_strokes_svg)


In [ ]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
image = sketch_app.init_canvas

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
response = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 3000,
                   use_cache = True, temperature = 0.5, min_p = 0.1)

<answer>
<formatting>
<concept>A car</concept>
<strokes>
<s1>
<points>'x50y50', 'x60y50', 'x60y50', 'x60y40', 'x60y40', 'x50y40', 'x50y40', 'x50y50'</points>
<t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
<id>car body rectangle</id>
</s1>
<s2>
<points>'x50y50', 'x60y60', 'x60y60', 'x60y50'</points>
<t_values>0.00,0.55,0.5,1.00</t_values>
<id>car roof triangle</id>
</s2>
<s3>
<points>'x50y50', 'x60y50', 'x60y50', 'x60y40', 'x60y40', 'x50y40', 'x50y40', 'x50y50'</points>
<t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
<id>car body rectangle</id>
</s3>
<s4>
<points>'x50y50', 'x60y60', 'x60y60', 'x60y50'</points>
<t_values>0.00,0.55,0.5,1.00</t_values>
<id>car roof triangle</id>
</s4>
<s5>
<points>'x55y45', 'x58y45', 'x58y45', 'x58y40', 'x58y40', 'x55y40', 'x55y40', 'x55y45'</points>
<t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
<id>car front window square</id>
</s5>
<s6>
<points>'x55y45', 'x58y45', 'x58y45', 'x58y40', 'x58y40', 'x55y40', 'x55y40', 'x55y45'

In [27]:
sketching_commands = """"
<answer>
<concept>Butterfly</concept>
<strokes>
<s1>
<points>'x20y20', 'x20y25', 'x20y30'</points>
<t_values>0.00, 0.50, 1.00</t_values>
<id>body</id>
</s1>
<s2>
<points>'x20y30', 'x15y35', 'x10y40'</points>
<t_values>0.00, 0.50, 1.00</t_values>
<id>left top wing</id>
</s2>
<s3>
<points>'x20y30', 'x25y35', 'x30y40'</points>
<t_values>0.00, 0.50, 1.00</t_values>
<id>right top wing</id>
</s3>
<s4>
<points>'x20y20', 'x15y15', 'x10y10'</points>
<t_values>0.00, 0.50, 1.00</t_values>
<id>left bottom wing</id>
</s4>
<s5>
<points>'x20y20', 'x25y15', 'x30y10'</points>
<t_values>0.00, 0.50, 1.00</t_values>
<id>right bottom wing</id>
</s5>
</strokes>
</answer>
"""



model_strokes_svg = sketch_app.parse_model_to_svg(sketching_commands)
current_direcotory = os.getcwd()
out_path = os.path.join(current_direcotory ,sketch_app.path2save, sketch_app.target_concept)
os.makedirs(out_path, exist_ok=True)


with open(f"{sketch_app.path2save}/{sketch_app.target_concept}.svg", "w") as svg_file:
    svg_file.write(model_strokes_svg)


# # vector->pixel
# # save the sketch to png with blank backgournd
# cairosvg.svg2png(url=f"{self.path2save}/{self.target_concept}.svg", write_to=f"{self.path2save}/{self.target_concept}.png", background_color="white")

# # save the sketch to png on the canvas
# output_png_path = f"{self.path2save}/{self.target_concept}_canvas.png"
# cairosvg.svg2png(url=f"{self.path2save}/{self.target_concept}.svg", write_to=output_png_path)
# foreground = Image.open(output_png_path)
# self.init_canvas.paste(Image.open(output_png_path), (0, 0), foreground)
# self.init_canvas.save(output_png_path)

In [ ]:
# 16bit quantized + using A100 + target concetp the same as contextual concept: house


start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"Started at: {start_time}")

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
image = sketch_app.init_canvas

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
response = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 3000,
                   use_cache = True, temperature = 0.5, min_p = 0.1)


end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"Ended at: {end_time}")


Started at: 2024-12-30 16:46:24
<answer>

<concept>House</concept>
<strokes>
    <s1>
        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>
        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
        <id>house base front rectangle</id>
    </s1>
    <s2>
        <points>'x24y27', 'x36y28', 'x36y28', 'x36y21', 'x36y21', 'x36y12', 'x36y12', 'x24y11'</points>
        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
        <id>house base right section</id>
    </s2>
    <s3>
        <points>'x13y27', 'x18y37','x18y37', 'x24y27'</points>
        <t_values>0.00,0.55,0.5,1.00</t_values>
        <id>roof front triangle</id>
    </s3>
    <s4>
        <points>'x18y37', 'x30y38', 'x30y38', 'x36y28'</points>
        <t_values>0.00,0.55,0.5,1.00</t_values>
        <id>roof right section</id>
    </s4>
    <s5>
        <points>'x26y25', 'x29y25', 'x29y25', 'x29y21', 'x29y21', 'x26y21', 'x26y21', 'x26y25'</points>
        <t_v

In [ ]:
sketching_commands = """"
<concept>House</concept>
<strokes>
    <s1>
        <points>'x13y27', 'x24y27', 'x24y27', 'x24y11', 'x24y11', 'x13y11', 'x13y11', 'x13y27'</points>
        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
        <id>house base front rectangle</id>
    </s1>
    <s2>
        <points>'x24y27', 'x36y28', 'x36y28', 'x36y21', 'x36y21', 'x36y12', 'x36y12', 'x24y11'</points>
        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
        <id>house base right section</id>
    </s2>
    <s3>
        <points>'x13y27', 'x18y37','x18y37', 'x24y27'</points>
        <t_values>0.00,0.55,0.5,1.00</t_values>
        <id>roof front triangle</id>
    </s3>
    <s4>
        <points>'x18y37', 'x30y38', 'x30y38', 'x36y28'</points>
        <t_values>0.00,0.55,0.5,1.00</t_values>
        <id>roof right section</id>
    </s4>
    <s5>
        <points>'x26y25', 'x29y25', 'x29y25', 'x29y21', 'x29y21', 'x26y21', 'x26y21', 'x26y25'</points>
        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
        <id>left window square</id>
    </s5>
    <s6>
        <points>'x31y25', 'x34y25', 'x34y25', 'x34y21', 'x34y21', 'x31y21', 'x31y21','x31y25'</points>
        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
        <id>right window square</id>
    </s6>
    <s7>
        <points>'x17y11', 'x17y18', 'x17y18', 'x21y18', 'x21y18', 'x21y11', 'x21y11', 'x17y11'</points>
        <t_values>0.00,0.3,0.25,0.5,0.5,0.75,0.75,1.00</t_values>
        <id>front door</id>
    </s7>
</strokes>

"""



model_strokes_svg = sketch_app.parse_model_to_svg(sketching_commands)
current_direcotory = os.getcwd()
out_path = os.path.join(current_direcotory ,sketch_app.path2save, sketch_app.target_concept)
os.makedirs(out_path, exist_ok=True)


with open(f"{sketch_app.path2save}/{sketch_app.target_concept}.svg", "w") as svg_file:
    svg_file.write(model_strokes_svg)


# # vector->pixel
# # save the sketch to png with blank backgournd
# cairosvg.svg2png(url=f"{self.path2save}/{self.target_concept}.svg", write_to=f"{self.path2save}/{self.target_concept}.png", background_color="white")

# # save the sketch to png on the canvas
# output_png_path = f"{self.path2save}/{self.target_concept}_canvas.png"
# cairosvg.svg2png(url=f"{self.path2save}/{self.target_concept}.svg", write_to=output_png_path)
# foreground = Image.open(output_png_path)
# self.init_canvas.paste(Image.open(output_png_path), (0, 0), foreground)
# self.init_canvas.save(output_png_path)

In [ ]:
!pwd

/content/CompetitiveSketchAgent


In [ ]:
sketch_app.target_concept

'car'